# FLIP (00): Data Science 
**(Module 04: Probability Theory)**

---
- Materials in this module include resources collected from various open-source online repositories.
- You are free to use,but NOT allowed to change and distribute this package.

Prepared by and for 
**Student Members** |
2006-2018 [TULIP Lab](http://www.tulip.org.au), Australia

---

## Session 1  Discrete sampling

In the last problem of the first session we sampled with given probabilities using prefix sums and binary search. The sampling code is the bottleneck of the whole process, its running times is 3 times higher than the code for uniform probabilities, and the difference would only increase for larger number of outcomes. In the next two problems we discuss two simple, but powerful techniques one can use to sample in time $O(1)$.

**Problem 2a.** Consider the problem of sampling with known probabilities $p_1,\ldots,p_d$. Suppose that you have a black-box that samples with probabilities $q_1,\ldots,q_d$ that are close to $p_1,\ldots,p_d$, say
$$ \forall_{i=1,\ldots,n} p_i \le (1+\varepsilon)q_i.$$

* How can you use this black-box to sample with probabilities $p_1,\ldots,p_d$? It is expected, that the running time of the algorithm would be non-deterministic.
* Prove that your algorithm is correct.
* Implement the algorithm and use it to give a faster implementation for **Problem 1c**.

In [ ]:
%%time

%matplotlib inline
import math
import random
import matplotlib.pyplot as plt
import numpy as np
import bisect

stats = np.loadtxt('birthday_us_births_69_88.csv', skiprows = 1, delimiter=',', dtype = int)

maxi = max(stats[:,2])
print(maxi)

def funkcja(v, tab):
    tab = [0 for i in range(0, v + 1)]
    k = 0
    while True:
        p = random.randint(0, v - 1)
        q = random.randint(0, maxi)
        if stats[p][2] >= q:
            #return p
            i = p;
            tab[i] = tab[i] + 1
            if tab[i] > 1:
                return k
            k = k + 1
    return -1

v = len(stats)
tab = [0 for i in range(0, v + 1)]
tab2 = [funkcja(v, tab) for i in range(100000)]

plt.hist(tab2, bins=30)
plt.show()

# ilość kroków aż będzie dana data
# samplować ale od dni a nie kiedy pierwsze wystąpienie


**Problem 2b.** One of the reasons this implementation is not significantly faster than the one in **Problem 1c** , apart from $d$ being rather small, is that we are using Python's interpreter a bit too much, and Python is slow. One way around this is usually to use a library function - **searchsorted** is much faster than an equivalent code implemented in pure Python. But even if the functionality you need is not implemented in a lower level language as
a library function, there is still hope. You can try to implement it using optimized array algebra, for example using **numpy**. In this problem, your task is to rewrite the previous algorithm, so that the amount of *looping* is reduced to a minimum necessary. In particular, you should create a *vectorized* version of random dates generation (in bulk), while the actual search for duplicates remains a loop with a **set**. Here are some useful tips:
   * You can perform arithmetic, comparisons, etc. on **numpy** arrays.
   * You can generate whole **numpy** arrays of random numbers at once.
   * You can even perform parallel look-up like in the example below.

In [ ]:

%%time

%matplotlib inline
import math
import random
import matplotlib.pyplot as plt
import numpy as np
import bisect

# saplowanie (tu: ile razy dany dzień się powtórzył)
# średnia ilość ile razy dzień się powtórzył

#X = np.array([10,3,7])
#I = np.array([1,1,2,2])
#print(X[I])
#X = np.array([[1,2],[3,4]])
#I = np.array([0,0,1])
#J = np.array([1,0,1])
#print(X[I,J])

stats = np.loadtxt('birthday_us_births_69_88.csv', skiprows = 1, delimiter=',', dtype = int)

maxi = max(stats[:,2])
#print(maxi)

N = 30000000
v = len(stats)

tabb = np.random.randint(0, v, N)
OK = np.random.randint(1, maxi + 1, N) <= stats[tabb, 2]
tabb = tabb[OK]
print(tabb)

k = 0

def funkcja(v, tab):
    global k
    start = k
    tab = [0 for i in range(0, v + 1)]
    while True:
        tab[tabb[k]] = tab[tabb[k]] + 1
        if tab[tabb[k]] > 1:
            return k - start
        k = k + 1
    return -1

v = len(stats)
tab = [0 for i in range(0, v + 1)]
tab2 = [funkcja(v, tab) for i in range(100000)]
    

plt.hist(tab2, bins=30)
plt.show()

# losuj 10k randomów (dni) z rozkładu jednostajnego
# druga tablica, czy zaakceptowane dni
# wyodrębnia te dni, które są zaakceptowane (mamy wysamplowane dni)



**Problem 2c (Squaring the histogram).** In this problem, we again want to sample with known probabilities $p_1,\ldots,p_n$, but this time we make no assumptions on $p_i$. Consider the following algorithm:
   * Let $V$ be the mean of $p_i$, i.e. $M=\frac{1}{n}$.
   * Create $n$ buckets, each with volume $V$, put each $p_i$ into a separate bucket.
   * Until there exists a bucket $A$ that is not full, find a bucket $B$ that overflows, and trasfer probability from $B$ to $A$ until $A$ is exactly full

Show that:
   * This algorithm always ends.
   * When it ends, each bucket contains pieces of exactly two $p_i$'s.

How to use the result of this algorithm to sample with probabilities $p_i$. Argue that your algorithm is correct and implement it. The sampling part should be *vectorized*. Use this algorithm to sample birthdates again, and test its efficiency.

In [ ]:
%%time
%matplotlib inline
import math
import random
import matplotlib.pyplot as plt
import numpy as np
import heapq
from operator import itemgetter

stats = np.loadtxt('birthday_us_births_69_88.csv', skiprows = 1, delimiter=',', dtype = int)
dane = stats[:,2]

size = dane.size
avg = np.average(dane)
avg = math.ceil(avg)

"""
sum = 0
for i in range(len(stats)):
    sum += stats[i][2]
nieuzyte = (avg * 372) - sum
"""

big = []
small = []
for i in range(size):
    if dane[i] > avg:
        big.append([dane[i], i])
    elif dane[i] <= avg:
        small.append([dane[i], i])

tab_min = small
tab_max = big
squared = np.zeros(shape=(size, 3), dtype=int)

k = 0
for i in range(size):
    if k == len(tab_max):
        pom = np.array([tab_min[i][0], tab_min[i][1], -1])
        squared[i] = pom
    else:
        pom = np.array([tab_min[i][0], tab_min[i][1], tab_max[k][1]])
        squared[i] = pom
        tab_max[k][0] -= (avg - tab_min[i][0])
        if tab_max[k][0] <= avg:
            pom2 = [tab_max[k][0], tab_max[k][1]]
            tab_min.append(pom2)
            k += 1

N = 100000
squared2 = squared.tolist()

"""
def funkcja():
    k = 0
    tab = [0 for i in range(size + 1)]
    while True:
        p = np.random.randint(0, size)
        q = np.random.randint(0, avg)
        w = squared2[p][0]
        if q > w:
            tab[squared2[p][2]] += 1
            if squared2[p][2] == -1:
                k += 1
                continue
            if tab[squared2[p][2]] > 1:
                return k
        else:
            tab[squared2[p][1]] += 1
            if tab[squared2[p][1]] > 1:
                return k
        k = k + 1
"""

tab1 = np.random.randint(0, size, 30 * N).tolist()
tab2 = np.random.randint(0, avg, 30 * N).tolist()
k = 0

def funkcja2():
    global k
    start = k
    tab = [0 for i in range(size + 1)]
    while True:
        p = tab1[k]
        q = tab2[k]
        w = squared2[p][0]
        if q > w:
            tab[squared2[p][2]] += 1
            if squared2[p][2] == -1:
                k += 1
                start += 1
                continue
            if tab[squared2[p][2]] > 1:
                return k - start
        else:
            tab[squared2[p][1]] += 1
            if tab[squared2[p][1]] > 1:
                return k - start
        k += 1

wyniki = [funkcja2() for i in range(N)]

plt.hist(wyniki, bins=30)
plt.show()


**Problem 2d.** Show that the frequency histogram for empirical birthday frequencies can actually be computed exactly, and implement your idea. To this end, design a recurence relation using conditional probabilities and use dynamic programming.